# Sum-of-Squares (SOS) optimization
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


## 배경(Background)
SOS (Sum-of-Squares) 최적화는 특별한 유형의 볼록 최적화이다. 이 유형의 최적화는 Drake의 MathematicalProgram에서 지원하는 다양한 솔버를 통해 해결할 수 있는 Semidefinite Programming (SDP) 문제로 공식을 만들 수 있다.

다항식 $p(x)$이 제곱의 합(sum-of-squares)이라면

$\begin{aligned}
p(x) = \sum_i q_i(x)^2
\end{aligned}$

"각각 $q_i(x)$가 또한 정해지지 않은(indeterminates) $x$의 다항식인 경우에, $p(x)$는 제곱의 합이라면 $p(x)$가 모든 $x$에 대해 음이 아닌 *충분* 조건이다. 어떤 경우에는 필요충분 조건이기도 하다. 다항식의 제곱의 합과 음이 아닌 관계에 대한 자세한 내용은 [Semidefinite Optimization and Convex Algebraic Geometry](http://www.mit.edu/~parrilo/sdocag/)의 3장을 참조할 수 있다.

다항식 p(x)가 합의 제곱이라는 것은 다음과 같은 양의 semidefinite (psd) 행렬 Q의 존재와 동일하다. 다음과 같이

$\begin{aligned}
p(x) = z(x)^T Q z(x), Q\succeq 0
\end{aligned}$

여기서 $z(x)$는 $x$의 특정 차수까지 모든 단항식을 포함한다. $x\in\mathbb{R}^n$ 와 $p(x)$의 차수가 $2d$라면, $z(x)$는 $x$의 최대 $d$차까지 모든 단항식을 포함한다. 즉, $z(x)$의 크기는 ${{n+d}\choose{d}}$이다. 따라서 제곱의 합 다항식 $p(x)$를 찾는 것은 추가 제약 조건으로 양의 semidefinite 행렬 $Q$를 찾는 것과 동일하다. 이 제약 조건은 다항식 $z(x)^TQz(x)$ 와 $p(x)$가 동일하다는 것이다 (동일한 계수).

예를 들어, 다항식

$\begin{aligned}
p(x) = 2x_1^4 + 5x_2^4 - 2x_1^2x_2^2 + 2x_1^3x_2 + 2x_1 + 2,
\end{aligned}$

다음과 같다.

$\begin{aligned}
p(x) = \underbrace{\begin{bmatrix} 1 \\x_1 \\x_2 \\ x_1^2 \\ x_1x_2 \\ x_2^2\end{bmatrix}^T}_{z(x)^T} \underbrace{\frac{1}{3}\begin{bmatrix} 6 & 3 & 0 & -2 & 0 & -2\\ 3 & 4 & 0 & 0 & 0 & 0\\0 & 0 & 4 & 0 & 0 & 0\\-2 & 0 & 0 & 6 & 3 & -4\\ 0 & 0 & 0 & 3 & 5 & 0\\ -2 & 0 & 0 & -4 & 0 & 15\end{bmatrix}}_{Q} \underbrace{\begin{bmatrix} 1 \\ x_1 \\x_2 \\x_1^2 \\x_1x_2\\ x_2^2\end{bmatrix}}_{z(x)}
\end{aligned}$

이 $Q$ 행렬은 양의 semidefinite이다. 따라서 $p(x)$는 제곱의 합 다항식이다. 실제로 $p(x)$는 다음과 같이 제곱의 합으로 분해될 수 있다.

$\begin{aligned}
p(x) = \frac{4}{3}x_2^2 + \frac{1349}{705}x_2^4 + \frac{1}{12}(4x_1+3)^2+\frac{1}{15}(3x_1^2+5x_1x_2)^2+\frac{1}{315}(-21x_1^2+20x_2^2+10)^2+\frac{1}{59220}(328x_2^2-235)^2
\end{aligned}$

## Solving sum-of-squares problem
제곱의 합 문제에서 우리는 종종 특정 추가 속성을 만족하는 합의 제곱 다항식을 찾고 싶어한다. 여기서 제곱의 합 문제에는 두 가지 유형의 변수가 있다는 점에 유의해야 한다. 미정계수(indeterminates)와 결정 변수(decision variables)이다.

* 결정 변수(Decision variables): 다항식의 계수는 우리의 결정 변수의 affine 함수이다. 우리는 결정 변수를 최적화한다.
* 미정계수(Indeterminates): 어떤 값도 취할 수 있는 변수. 위의 예에서 $x$는 미정계수(indeterminates)이다. 우리가 작성하는 제약 조건은 모든 $x$에 대해 유지되어야 한다. 즉, 우리는 이러한 미정계수를 최적화하지 않는다.

Drake의 MathematicalProgram 내에서 제곱의 합 최적화에서 일반적으로 사용되는 몇 가지 함수가 있다. 여기서 몇 가지를 나열하자.
* `MathematicalProgram.NewIndeterminates`: 새로운 미정계수(indeterminates)를 선언하고 반환한다.
* `MathematicalProgram.AddSosConstraint`: 다항식에 제곱의 합(SOS) 제약 조건을 부과한다.
* `MathematicalProgram.NewSosPolynomial`: 새로운 다항식을 생성하고 반환한다. 이 다항식에 SOS 제약을 부과한다.
* `MathematicalProgram.NewFreePolynomial`: 새로운 다항식을 생성하고 반환한다. 아직 이 다항식에 어떠한 제약 조건도 부과되지 않은 상태다.
* `MathematicalProgram.AddEqualityConstraintBetweenPolynomials`: 두 다항식이 동일하다는 선형 제약 조건을 부여한다 (일치하는 단항식에 대해 동일한 계수를 갖음).

각 함수의 상세 내용은 나중에 설명하겠다. 먼저 몇 가지 예시를 살펴보자.

### Imports

In [ ]:
import math

import numpy as np
import matplotlib.pyplot as plt

from pydrake.solvers import MathematicalProgram, Solve
import pydrake.symbolic as sym

### Example 1
예를 들어, 다음 최적화 문제(Example 3.50 of [Semidefinite Optimization and Convex Algebraic Geometry](http://www.mit.edu/~parrilo/sdocag/))를 풀고 싶다면

$\begin{aligned}
\min_{a}& -a_0 - a_1\\
\text{s.t }& x^4 + a_0 x + 2 + a_1 \text{ is SOS}\\
&(a_0 - a_1 + 1)x^2 + a_1x + 1\text{ is SOS}
\end{aligned}$

이 경우 $x$는 미정계수이고 $a$는 결정 변수이다.

아래 코드는 Drake의 MathematicalProgram에서 이 SOS 문제를 모델링하고 해결하는 방법을 보여준다.

In [ ]:
"""
Code for example 1.
"""

# Initialize an empty optimization program.
prog = MathematicalProgram()
# Declare "a" as decision variables.
a = prog.NewContinuousVariables(2)
# Declare "x" as indeterminates
x = prog.NewIndeterminates(1)

# Declare p1(x) = pow(x, 4) + a0 * x + 2 + a1. The second argument
# [x] indicates that the indeterminates in the polynomial is x.
p1 = sym.Polynomial(x[0]**4 + a[0] * x[0] + 2 + a[1], [x])
# Declare p2(x) = (a0 - a1 + 1) * pow(x, 2) + a1 * x + 1. The second argument
# [x] indicates that the indeterminates in the polynomial is x.
p2 = sym.Polynomial((a[0] - a[1] + 1) * x[0] * x[0] + a[1] * x[0] + 1, [x])
# Add the constraint that p1(x) and p2(x) are SOS
prog.AddSosConstraint(p1)
prog.AddSosConstraint(p2)

# Add the cost -a0 - a1
prog.AddCost(-a[0] - a[1])

# Solve the problem.
result = Solve(prog)

# Retrieve the solution.
print(result.get_solution_result())
print("a =", result.GetSolution(a))

### Example 2

$\begin{aligned}
\min_{a, b, c}&\; a + b + c\\
\text{s.t }& x^4 + ax^3 + bx^2 + cx + 1 \ge 0 \quad\forall x\in[0, 1]
\end{aligned}$

이 문제를 해결하기 위해 우리는 [Semidefinite Optimization and Convex Algebraic Geometry](http://www.mit.edu/~parrilo/sdocag/)의 3.72 정리의 특수 경우를 사용한다. :
짝수 차수의 단변수 다항식 $p(x)$가 구간 $[0, 1]$에서 음이 아닌 것은 아래와 같이 표현될 수 있다.

$\begin{aligned}
p(x) = s(x) + x(1-x)t(x)
\end{aligned}$

여기서 $s(x), t(x)$는 모두 제곱의 합이다.
따라서 문제를 다음과 같이 재정의할 수 있다.

$\begin{aligned}
\min_{a, b, c, t}&\; a+b+c\\
\text{s.t }& x^4 + ax^3 + bx^2 + cx + 1 - x(1-x)t(x) \text{ is SOS}\\
& t(x) \text{ is SOS}
\end{aligned}$

여기서 $x$는 미정계수이고, $a, b, c$ 그리고 다항식 $t(x)$의 계수들은 결정 변수이다. 아래 코드는 Drake의 MathematicalProgram으로 이 문제를 모델링하고 해결하는 방법을 보여준다.

In [ ]:
"""
Code for example 2.
"""

# Initialize an empty optimization program.
prog = MathematicalProgram()
# Declares indeterminates
x = prog.NewIndeterminates(1)

# Declares decision variable a, b, and c
a = prog.NewContinuousVariables(1)[0]
b = prog.NewContinuousVariables(1)[0]
c = prog.NewContinuousVariables(1)[0]

# Declare p(x), the second argument indicates this polynomial p(x) has x as the
# indeterminates.
poly_p = sym.Polynomial(x[0]**4 + a * x[0]**3 + b * x[0]** 2 + c * x[0] + 1, [x])
# Declares SOS polynomial t(x), such that s(x) = p(x) - x(1-x) * t(x) is SOS.
# The second return argument of NewSosPolynomial is the positive-semidefinite
# Gramian matrix of poly_s. We ignore this matrix here.
# t(x) should be a quadratic polynomial of x, hence the second input argument is
# 2, indicating the degree of the returned polynomial.
poly_t, _ = prog.NewSosPolynomial(sym.Variables(x), 2)
# Compute s(x) = p(x) - x(1-x)t(x) as a polynomial of indeterminate x.
poly_s = poly_p - sym.Polynomial(x[0] * (1 - x[0]), sym.Variables(x)) * poly_t
prog.AddSosConstraint(poly_s)

# Add the cost a + b + c
prog.AddCost(a + b + c)

result = Solve(prog)
print("Is success? ", result.is_success())
print(result.get_solution_result())
a_val = result.GetSolution(a)
b_val = result.GetSolution(b)
c_val = result.GetSolution(c)

# Now plot the polynomial p(x), that is always non-negative in the range [0, 1]
x_val = np.linspace(0, 1, 100)
y_val = np.power(x_val, 4) + a_val * np.power(x_val, 3) + b_val * np.power(x_val, 2) + c_val * x_val + 1
plt.xlabel("x")
plt.ylabel("p(x)")
plt.plot(x_val, y_val)

## 추가로 읽을꺼리(Further readings)

SOS 최적화는 제어 이론, 로봇공학, 양자 정보 이론 등 다양한 분야에서 널리 사용된다. 몇 가지 참고 자료는 다음과 같다.

#### SOS 이론(Sum-of-squares theory)
* [Semidefinite Optimization and Convex Algebraic Geometry](http://www.mit.edu/~parrilo/sdocag/) by G Blekherman, P Parrilo and R Thomas
* [Sum of squares techniques and polynomial optimization](http://www.princeton.edu/~amirali/Public/Presentations/CDC_2016_Parrilo_1) Presentation slides by Pablo Parrilo

#### 시스템 제어 이론(System control theory)
* [Structured Semidefinite Programs and Semialgebraic Geometry Methods in Robustness and Optimization](http://www.mit.edu/~parrilo/pubs/files/thesis.pdf) Pablo Parrilo's PhD thesis
* [Robust Region-of-Attraction Estimation](https://ieeexplore.ieee.org/abstract/document/5337881) Ufuk Topcu, Andrew Packard, Peter Seiler, Gary Balas, IEEE Transactions on Automatic Control, 2009

#### Robotics
* [LQR-trees: Feedback motion planning via sums-of-squares verification](http://groups.csail.mit.edu/robotics-center/public_papers/Tedrake10.pdf) Russ Tedrake, Ian Manchester, Mark Tobenkin, John Roberts, International Journal of Robotics Research, 2010
* [Funnel libraries for real-time robust feedback motion planning](http://groups.csail.mit.edu/robotics-center/public_papers/Majumdar16.pdf) Anirudha Majumdar and Russ Tedrake, International Journal of Robotics Research, 2017